In [ ]:
%matplotlib inline
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import datetime
from datetime import timedelta
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (30, 15)
# mpl.rcParams['axes.grid'] = False

target="close"

In [ ]:
def getDataset(full=False):
    url = "./datasets/DAT_ASCII_EURUSD_M1_2018.csv"
    df = pd.read_csv(url, names=list(["date","open", "high", "low", "close", "volume"]), header=None, sep=";")
    # df.drop(columns=['open', 'high', 'low'], inplace=True)
    pdlen=len(df)
    
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    # df.set_axis(df['date'], inplace=True)
    # df.drop(columns=["volume"], inplace=True)
    df.drop(columns=["volume", "low", "open", "high"], inplace=True)
    # df["Close"]= df["close"] # compatibility
    # df["Volume"]= df["volume"] # compatibility
    # df.drop(columns=['close'], inplace=True)
    # df["ewma_close"]=df["close"].ewm(4).mean()
    if full is False:
        df=df[58::60]
        
    return df

In [ ]:
df = getDataset(full=False)

# df.set_axis(df['date'])[target].plot(figsize=(15,8))
# plt.legend(["close"])
# plt.grid(True)
# plt.title("Wykres przedstawiający wartość stosunku ceny euro do dolara amerykańskiego w roku 2018")
# plt.xlabel("Data")
# plt.ylabel("Wartość stosunku euro do dolara")

In [ ]:
date_time = pd.to_datetime(df.pop('date'))
print(date_time)

In [ ]:
# adjusted_date_time = []
# print([date_time.values[0]+timedelta(minutes=(i-1)*1) for i in range(len(date_time.values))])


date_time_clone = date_time.copy()
new_values=pd.date_range(date_time.values[0], periods=len(date_time), freq='1h')

i=0
for key in date_time_clone.copy().keys():
    date_time_clone[key]=new_values[i]
    i=i+1

date_time=date_time_clone
print(date_time)

In [ ]:
timestamp_s = date_time.map(datetime.datetime.timestamp)
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

timestamp_s[:5]

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def plotLossHistory(history, figsize=(30,15)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(history.history['loss'], label='Wartość funkcji kosztu dla zbioru treningowego')
    ax.plot(history.history['val_loss'], label='Wartość funkcji kosztu dla zbioru walidacyjnego')
    plt.ylabel('Wartość funkcji kosztu')
    plt.xlabel('Numer epoki')
    ax.legend(loc="upper left")
    plt.title("Wartość funkcji kosztu na przestrzeni epok")
    plt.show()

In [ ]:
def plotMAEHistory(history):
    fig, ax = plt.subplots(figsize=(30,15))
    ax.plot(history.history['mean_absolute_error'], label='MAE podczas treningu')
    ax.plot(history.history['val_mean_absolute_error'], label='MAE na zbiorze walidacyjnym')
    plt.ylabel('MAE value')
    plt.xlabel('Numer epoki')
    ax.legend(loc="upper left")
    plt.show()

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def plot(self, model=None, plot_col=target, max_subplots=3, title=None):
  inputs, labels = self.example
  # plt.figure(figsize=(15, 15))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)

    if title is not None and n==0:
      plt.title(title)

    plt.xticks(np.arange(0, 48, step=1))
    # plt.xticks(np.arange(0, len(self.input_indices)+1, step=1))
    
    plt.ylabel(f'{plot_col} [znormalizowana wartość]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Wejście', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Prawdziwe wartości', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      # print(predictions)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predykcje',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Czas od pierwszej próbki [h]')

WindowGenerator.plot = plot

In [ ]:
# w2.plot()

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=[target])
single_step_window

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each timestep is the input
    # from the previous time step plus the delta
    # calculated by the model.
    return inputs + delta

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [ ]:
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    label_columns=[target])

wide_window

In [ ]:
# print('Input shape:', single_step_window.example[0].shape)
# print('Output shape:', linear(single_step_window.example[0]).shape)

In [ ]:
MAX_EPOCHS = 30

def compile_and_fit(model, window, patience=5, max_epochs=MAX_EPOCHS):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=max_epochs,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
mpl.rcParams['figure.figsize'] = (20, 10)

In [ ]:
import time

def exper(n=10):
    start = time.time()

    print(f"n={n}")

    rnn_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(n, return_sequences=True),
        tf.keras.layers.LSTM(n, return_sequences=True),
        tf.keras.layers.LSTM(n, return_sequences=True),
        tf.keras.layers.LSTM(n, return_sequences=True),
        tf.keras.layers.Dense(1)
    ])

    history = compile_and_fit(rnn_model, wide_window, max_epochs=50, patience=50)

    IPython.display.clear_output()

    end = time.time()
    print("validation")
    val_performance_single['RNN'] = rnn_model.evaluate(wide_window.val, verbose=1)
    vali=val_performance_single['RNN']
    print("test")
    performance_single['RNN'] = rnn_model.evaluate(wide_window.test, verbose=1)
    testi=performance_single['RNN']

    ret = f"{n} & {end-start:0.3f} & {vali[0]:0.4f} & {vali[1]:0.4f} & {testi[0]:0.4f} & {testi[1]:0.4f} \\\\"

    return ret, rnn_model, history

# ret, model, history = exper(n=240)
# ret, model40, history = exper(n=40)

# print(ret)
# 20,30,40,50,60,80,120,240
#5,10,20,30,40,50,60,80,120 
# xd = [exper(i)[0] for i in np.array([5,10,20,30,40,50,60,80,120,240])]
# print(xd)


# wide_window.plot(model)

In [ ]:
def plotMany(self, model=None, plot_col=target, max_subplots=3, title=None, additionalModels=None):
  inputs, labels = self.example
  # plt.figure(figsize=(15, 15))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)

    if title is not None and n==0:
      plt.title(title)

    plt.xticks(np.arange(0, len(self.input_indices)+1, step=1))
    
    plt.ylabel(f'{plot_col} [znormalizowana wartość]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Wejście', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index
 
    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Prawdziwe wartości', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
  
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predykcje',
                  c='#ff7f0e', s=64)
    if additionalModels is not None:
        for aModel in additionalModels:
            predictions = aModel(inputs)
            plt.scatter(self.label_indices, predictions[n, :, label_col_index], label='Predykcjee', s=64)       
    if n == 0:
      plt.legend()

  plt.xlabel('Czas od pierwszej próbki [h]')

WindowGenerator.plotMany = plotMany

In [ ]:
# wide_window.plotMany(model20, additionalModels=np.array([model40]))

In [ ]:
# plotLossHistory(history, figsize=(20,8))

In [ ]:
# plotMAEHistory(history)

In [ ]:
# for name, value in performance_single.items():
#   print(f'{name:12s}: {value[1]:0.4f}')

In [ ]:
baseline = Baseline()
baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

In [ ]:
val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(wide_window.val)
performance['Baseline'] = baseline.evaluate(wide_window.test, verbose=0)

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_window

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.val, verbose=0)
multi_window.plot(last_baseline)

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.losses.MeanSquaredError(),
                        metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance['Repeat'] = repeat_baseline.evaluate(multi_window.val)
multi_performance['Repeat'] = repeat_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(repeat_baseline)

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(24, return_sequences=True),
    tf.keras.layers.LSTM(24, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.train, verbose=0)
multi_window.plot(multi_lstm_model)

In [ ]:
multi_gru_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.GRU(24, return_sequences=True),
    tf.keras.layers.GRU(24, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_gru_model, multi_window)

IPython.display.clear_output()

multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val)
multi_performance['GRU'] = multi_gru_model.evaluate(multi_window.train, verbose=0)
multi_window.plot(multi_gru_model)

In [159]:
import time

val_performance_single={}
performance_single={}
def exper(n=10, layerType=tf.keras.layers.LSTM):
    start = time.time()

    print(f"n={n}")

    rnn_model = ResidualWrapper(tf.keras.Sequential([
        tf.keras.layers.GRU(n, return_sequences=True),
        tf.keras.layers.GRU(n, return_sequences=False),
        # tf.keras.layers.LSTM(n, return_sequences=True),
        # tf.keras.layers.LSTM(n, return_sequences=True),
        tf.keras.layers.Dense(OUT_STEPS,kernel_initializer=tf.initializers.zeros),
        tf.keras.layers.Reshape([OUT_STEPS, 1])
    ]))

    history = compile_and_fit(rnn_model, multi_window, max_epochs=50, patience=50)

    IPython.display.clear_output()

    end = time.time()
    print("validation")
    val_performance_single['RNN'] = rnn_model.evaluate(multi_window.val, verbose=1)
    vali=val_performance_single['RNN']
    print("test")
    performance_single['RNN'] = rnn_model.evaluate(multi_window.test, verbose=1)
    testi=performance_single['RNN']

    ret = f"{n} & {end-start:0.3f} & {vali[0]:0.4f} & {vali[1]:0.4f} & {testi[0]:0.4f} & {testi[1]:0.4f} \\\\"

    return ret, rnn_model, history

# ret, model, history = exper(n=120)
# ret, model40, history = exper(n=40)

# print(ret)
# 20,30,40,50,60,80,120,240
#5,10,20,30,40,50,60,80,120 
xd = [exper(i, layerType=tf.keras.layers.LSTM)[0] for i in np.array([5,10,20,30,40,50,60,80,120,240])]
print(xd)


# multi_window.plot(model)

validation
38/38 [==============================] - 0s 7ms/step - loss: 0.0047 - mean_absolute_error: 0.0438
test
18/18 [==============================] - 0s 7ms/step - loss: 0.0042 - mean_absolute_error: 0.0423
['5 & 61.246 & 0.0046 & 0.0428 & 0.0053 & 0.0498 \\\\', '10 & 61.891 & 0.0100 & 0.0734 & 0.0344 & 0.1651 \\\\', '20 & 64.311 & 0.0052 & 0.0470 & 0.0104 & 0.0830 \\\\', '30 & 65.813 & 0.0062 & 0.0564 & 0.0192 & 0.1236 \\\\', '40 & 65.965 & 0.0056 & 0.0527 & 0.0094 & 0.0822 \\\\', '50 & 69.623 & 0.0049 & 0.0442 & 0.0047 & 0.0458 \\\\', '60 & 73.377 & 0.0048 & 0.0442 & 0.0064 & 0.0598 \\\\', '80 & 72.527 & 0.0059 & 0.0545 & 0.0061 & 0.0575 \\\\', '120 & 90.961 & 0.0049 & 0.0443 & 0.0045 & 0.0435 \\\\', '240 & 150.378 & 0.0047 & 0.0438 & 0.0042 & 0.0423 \\\\']
